# Sonic Frontiers Region-Based Convolutional Neural Network
A model for predicting the class label and bounding box location of objects within screenshots of the video game Sonic Frontiers.

# Model Preparation

## Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os

In [2]:
sonicDF = pd.read_csv("SonicFrontiersData.txt", sep = ", ", engine = "python")
sonicDF.head()

,Image,X,Y,Width,Height,Label
0,SonicFrontiers1.jpg,303.333333,327.833333,353.500000,408.916667,Sonic
1,SonicFrontiers10.jpg,296.545455,383.090909,374.181818,488.727273,Sonic
2,SonicFrontiers100.jpg,304.818182,327.090909,385.000000,404.727273,Sonic
3,SonicFrontiers1000.jpg,250.250000,247.333333,319.666667,316.750000,Sonic
4,SonicFrontiers1001.jpg,278.923077,321.461538,337.076923,394.153846,Sonic


In [3]:
sonicDF.dtypes

Image      object
X         float64
Y         float64
Width     float64
Height    float64
Label      object
dtype: object

In [4]:
sonicDF.shape

(2036, 6)

## Cleaning the Data

In [5]:
sonicDF = sonicDF.drop_duplicates(ignore_index = True)
sonicDF = sonicDF.sample(frac = 1, random_state = 7, ignore_index = True)
sonicDF.head()

,Image,X,Y,Width,Height,Label
0,SonicFrontiers2421.jpg,298.200000,196.700000,358.400000,320.600000,Sonic
1,SonicFrontiers1934.jpg,137.200000,380.800000,204.400000,448.700000,Rings
2,SonicFrontiers2197.jpg,280.700000,265.300000,352.800000,326.200000,Sonic
3,SonicFrontiers2020.jpg,389.083333,177.333333,402.500000,199.500000,Rings
4,SonicFrontiers1269.jpg,287.000000,312.083333,341.833333,394.333333,Sonic


In [6]:
sonicDF.shape

(2032, 6)

## Visualizing the Data

In [7]:
datasetPath = "D:\Sonic-Frontiers-Dataset"
imagePath = sonicDF["Image"][0]
print(imagePath)

SonicFrontiers2421.jpg
